In [1]:
import os
import numpy as np
import h5py
from scipy import stats
import scipy.io
import mne

mne.set_log_level('error')

from random import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchsummary import summary

import optuna


from utils.load import Load
from config.default import cfg

%load_ext autoreload
%autoreload 2


In [2]:
torch.manual_seed(42)
np.random.seed(42)


In [3]:
subject_id = 0

In [4]:
device_name = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_name)
print(device)

cuda


In [5]:
# Load the data  from the HDF5 file
target_dir = 'features'
tag = 'reproduced_with_bad'
file_path = os.path.join(target_dir, tag+'_'+cfg['subjects'][subject_id] + '.h5')


data = {}
with h5py.File(file_path, 'r') as h5file:
    for key in h5file.keys():
        data[key] = np.array(h5file[key])

# Time first [Time, Channels, Features]
for key, value in data.items():
    data[key] = np.transpose(value, (0, 2, 1, 3))

# Print the loaded data dictionary
for key, value in data.items():
    print(key, value.shape)

index (50, 26, 158, 2)
little (50, 26, 158, 2)
middle (50, 26, 158, 2)
ring (50, 26, 158, 2)
thumb (50, 26, 158, 2)


In [6]:
X = np.concatenate(list(data.values()), axis=0)
y = np.concatenate([np.ones(data[finger].shape[0]) * i for i, finger in enumerate(data)], axis=0)

# Normalize the data
# orig_shape = X.shape
# X = X.reshape(X.shape[0], -1)
# scaler = StandardScaler()
# X = scaler.fit_transform(X)
# X = X.reshape(orig_shape)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)





class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

        self.X = torch.from_numpy(X).float().to(device)
        self.y = torch.from_numpy(y).long().to(device)


    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        features = self.X[idx]
        label = self.y[idx]
        
        return features, label

train_dataset = CustomDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)

test_dataset = CustomDataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=True)

In [7]:
for features, label in train_dataloader:
    print(features.shape)
    print(label)
    break

torch.Size([10, 26, 158, 2])
tensor([2, 3, 1, 0, 0, 2, 4, 0, 3, 2], device='cuda:0')


In [8]:
def accuracy(dataloader):
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for features,  labels in dataloader:

            outputs = model(features)
            _, predicted = torch.max(outputs.data, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    accuracy = correct_predictions / total_predictions

    return accuracy * 100

In [9]:
class SingleLayerMLP(nn.Module):
    def __init__(self, time_stamps, channels, rnn_hidden_size,  output_size, activation):
        super(SingleLayerMLP, self).__init__()

        self.channels = channels
        self.fc1 = nn.Linear(2, 1)
        self.rnn = nn.RNN(channels, rnn_hidden_size, batch_first=True)
        self.lstm = nn.LSTM(channels, rnn_hidden_size, bias = False, batch_first=True, bidirectional=True)
        self.activation = nn.ReLU()
        self.fc2 = nn.Linear(rnn_hidden_size*2, output_size)
        #self.fc2 = nn.Linear(rnn_hidden_size*time_stamps, output_size)

        self.dropout = nn.Dropout(0.1)
        
        
    def forward(self, x):

        ###
        ###Merge Mu and Beta band power features
        ###
        # A version
        x = x.reshape(-1,2)
        x = self.fc1(x)         # Apply the linear layer
        x = x.view(-1, time_stamps, channels)         # Reshape the tensor back to its original shape: (26, 158)
        
        # B version
        #x = torch.mean(x, -1)

        # C version
        #x = x.reshape(10, -1, self.channels*2)

        # RNN
        #x, hn = self.rnn(x)
        x, (hn, cn) = self.lstm(x)
        x = x[:, -1, :]
       
        
        x = self.activation(x)
        x = self.dropout(x)

        x = self.fc2(x)

        return x



In [10]:
num_epochs = 100
learning_rate = 1e-3
rnn_hidden_size = 8



time_stamps = X.shape[1]
channels = X.shape[2]
model = SingleLayerMLP(time_stamps, channels, rnn_hidden_size, 5, nn.ReLU())
#summary(model, input_size=(X[0].shape));
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
def train(num_epochs, verbose = False):
    # single train
    model.train()
    for epoch in range(num_epochs):

        epoch_loss = 0.0
        for X, y in train_dataloader:
            optimizer.zero_grad()
            outputs = model(X)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        if  verbose and (epoch+1) % 10 == 0:
            train_accuracy = accuracy(train_dataloader)
            test_accuracy = accuracy(test_dataloader)
            print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss}, Train accuracy: {train_accuracy:.2f}%, Test accuracy: {test_accuracy:.2f}%")

    return accuracy(test_dataloader)

train(num_epochs, verbose=True)

Epoch 10/100, Loss: 32.04113698005676, Train accuracy: 27.00%, Test accuracy: 20.00%
Epoch 20/100, Loss: 30.964401721954346, Train accuracy: 29.50%, Test accuracy: 26.00%
Epoch 30/100, Loss: 30.100576996803284, Train accuracy: 37.50%, Test accuracy: 34.00%
Epoch 40/100, Loss: 28.323060870170593, Train accuracy: 44.00%, Test accuracy: 30.00%
Epoch 50/100, Loss: 26.627503633499146, Train accuracy: 54.00%, Test accuracy: 32.00%
Epoch 60/100, Loss: 25.095929861068726, Train accuracy: 53.00%, Test accuracy: 34.00%
Epoch 70/100, Loss: 22.142316162586212, Train accuracy: 62.00%, Test accuracy: 34.00%
Epoch 80/100, Loss: 20.731123089790344, Train accuracy: 68.50%, Test accuracy: 34.00%
Epoch 90/100, Loss: 19.58177238702774, Train accuracy: 66.50%, Test accuracy: 28.00%
Epoch 100/100, Loss: 17.375755786895752, Train accuracy: 78.00%, Test accuracy: 34.00%


26.0

In [12]:

def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    num_epochs = trial.suggest_int("num_epochs", 20, 500)
    hidden_size = trial.suggest_int("hidden_size", 4, 32)
    activation_name = trial.suggest_categorical("activation", ["relu", "elu", "leaky_relu"])
    optimizer = trial.suggest_categorical("optimizer", ["SGD", "Adam"])

    if activation_name == "relu":
        activation = nn.ReLU()
    elif activation_name == "elu":
        activation = nn.ELU()
    elif activation_name == "leaky_relu":
        activation = nn.LeakyReLU()

    if optimizer == "SGD":
        optimizer = optim.SGD
    elif optimizer == "Adam":
        optimizer = optim.Adam


    model = SingleLayerMLP(time_stamps=time_stamps, channels=channels, rnn_hidden_size=hidden_size, output_size=5, activation=activation)
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optimizer(model.parameters(), lr=learning_rate)
    return train(num_epochs)


In [13]:
# Hyperparameter tuning
n_trials = 10

study = optuna.create_study(direction="maximize")
study.optimize(lambda trial: objective(trial), n_trials=n_trials)

best_trial = study.best_trial

print(f'Best trial params: {best_trial.params}')
print(f'Best trial accuracy: {best_trial.value}%')

[I 2023-04-23 20:56:09,649] A new study created in memory with name: no-name-76a45959-612e-430c-b0e4-f38675eb26c4
[I 2023-04-23 20:56:35,840] Trial 0 finished with value: 30.0 and parameters: {'learning_rate': 0.06784288046287168, 'num_epochs': 440, 'hidden_size': 21, 'activation': 'leaky_relu', 'optimizer': 'SGD'}. Best is trial 0 with value: 30.0.
[I 2023-04-23 20:56:50,558] Trial 1 finished with value: 26.0 and parameters: {'learning_rate': 0.050061226309436296, 'num_epochs': 268, 'hidden_size': 21, 'activation': 'leaky_relu', 'optimizer': 'Adam'}. Best is trial 0 with value: 30.0.
[I 2023-04-23 20:56:55,500] Trial 2 finished with value: 32.0 and parameters: {'learning_rate': 0.03650758716825999, 'num_epochs': 82, 'hidden_size': 23, 'activation': 'relu', 'optimizer': 'Adam'}. Best is trial 2 with value: 32.0.
[I 2023-04-23 20:57:16,089] Trial 3 finished with value: 22.0 and parameters: {'learning_rate': 0.007884670395349756, 'num_epochs': 329, 'hidden_size': 9, 'activation': 'relu',

Best trial params: {'learning_rate': 0.03650758716825999, 'num_epochs': 82, 'hidden_size': 23, 'activation': 'relu', 'optimizer': 'Adam'}
Best trial accuracy: 32.0%
